# Исследование надежности заемщиков


**Цель исследования** - выявить закономерности, определяющие надёжность заёмщиков.

**План исследования**
1. Зависимость между количеством детей и возвратом кредита в срок.
2. Зависимость между семейным положением и возвратом кредита в срок.
3. Зависимость между уровнем дохода и возвратом кредита в срок.
4. Влияние разных целей кредита влияют на его возврат в срок.


## Предобработка данных

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/data.csv')      
except:
    data = pd.read_csv('/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
# доля пропусков
data.isna().mean()*100

children             0.000000
days_employed       10.099884
dob_years            0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        10.099884
purpose              0.000000
dtype: float64

В двух столбцах есть пропущенные значения: `days_employed` и `total_income`. 

`total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце можно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [6]:
data['total_income'] = data['total_income'].fillna(data.groupby(['income_type'])['total_income'].transform('median'))

### Обработка аномальных значений

В значениях стажа есть отрицательное количество дней. Заменим все отрицательные значения положительными.

In [7]:
data['days_employed'] = data['days_employed'].abs()

Медианное значение трудового стажа `days_employed` в днях.

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть.

Перечень уникальных значений столбца `children`.

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Проверим удаление артефактов.

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Контроль удаления пропусков.

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный.

In [14]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработка неявных дубликатов в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведём их к нижнему регистру.

In [15]:
data['education'] = data['education'].str.lower()

Количество строк-дубликатов в данных.

In [16]:
data.duplicated().sum()

71

Удалим дубликаты.

In [17]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [18]:
# функция categorize_income()
def categorize_income(total_income):
    '''функция присвоения категорий'''
    try:
        if total_income <= 30000:
            return 'E'
        if total_income <= 50000:
            return 'D'
        if total_income <= 200000:
            return 'C'
        if total_income <= 1000000:
            return 'B'
        if total_income > 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Перечень уникальных целей взятия кредита из столбца `purpose`.

In [20]:
data['purpose'].value_counts()

свадьба                                   790
на проведение свадьбы                     763
сыграть свадьбу                           760
операции с недвижимостью                  672
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   649
операции с жильем                         647
операции с коммерческой недвижимостью     645
жилье                                     641
покупка жилья                             640
покупка жилья для семьи                   637
недвижимость                              631
строительство собственной недвижимости    628
операции со своей недвижимостью           623
строительство жилой недвижимости          620
строительство недвижимости                619
покупка своего жилья                      619
покупка недвижимости                      616
ремонт жилью                              604
покупка жилой недвижимости                602
на покупку своего автомобиля              504
заняться высшим образованием      

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:
- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [21]:
# функция categorize_purpose()
def categorize_purpose(purpose):
    '''функция присвоения категории по цели'''
    try:
        if purpose.find('автомоби') != -1:
            return 'операции с автомобилем'
        if purpose.find('недвижим') != -1:
            return 'операции с недвижимостью'
        if purpose.find('жиль') != -1:
            return 'операции с недвижимостью'
        if purpose.find('свадь') != -1:
            return 'проведение свадьбы'
        if purpose.find('образован') != -1:
            return 'получение образования'
    except:
        pass
    return purpose


In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


### Исследование данных

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

<p>План решения:</p>
<ol>
<li>исследовать информацию о долге;</li>
<li>объединить заёмщиков по количеству детей;</li>
<li>исследовать количество заёмщиков в каждой группе;</li>
<li>вычислить % возвращенных кредитов в каждой группе и сравнить.</ol>

**Исследование информации о долге**

In [23]:
data['debt'].unique()

array([0, 1], dtype=int64)

Результат: значения предствалены в виде чисел. Полагаю, 1 - есть долг, 0 - нет долга.

**Объединение заёмщиков в группы по количеству детей, оценка данных**

In [24]:
data_grouped_by_children = data.groupby('children').agg({'debt': ['count', 'sum', 'mean']}).droplevel(0, axis=1).reset_index()
data_grouped_by_children['mean'] = data_grouped_by_children['mean']*100 #перевод в % для повышения читаемости
data_grouped_by_children = data_grouped_by_children.rename({'mean':'debt_percent'}, axis=1)
data_grouped_by_children['perсent_of_total'] = data_grouped_by_children['count'] / data['children'].count()*100
data_grouped_by_children.round(1)

,children,count,sum,debt_percent,perсent_of_total
0,0,14091,1063,7.5,66.1
1,1,4808,444,9.2,22.5
2,2,2052,194,9.5,9.6
3,3,330,27,8.2,1.5
4,4,41,4,9.8,0.2
5,5,9,0,0.0,0.0


**Вывод:** основную массу заёмщиков составляют бездетные и люди с одним ребёнком, суммарно 88,6%. Выборка по заёмщикам с 3 и более детьми мала, данные в них можно не учитывать. <p> Доля должников среди бездетных составляет 7,5% - это ниже, чем среди заёмщиков с 1 и 2 детьми (9,2% и 9,5% соответственно). </p>

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

План решения:<ol>
<li>исследование значений графы семейного положения</li>
<li>категоризация, расчет количества долгов по категориям</li>
<li>расчет % долга в каждой категории.</ol>

**Исследование значений графы семейного положения**

In [25]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [26]:
data['family_status'] = data['family_status'].str.lower() #значения к нижнему регистру

Результат: аномалий нет. 

**Категоризация, расчет и оценка данных**

In [27]:
data_grouped_by_family_status = data.groupby('family_status').agg({'debt': ['count', 'sum', 'mean']}).droplevel(0, axis=1).reset_index()
data_grouped_by_family_status['mean'] = data_grouped_by_family_status['mean']*100 
data_grouped_by_family_status = data_grouped_by_family_status.rename({'mean':'debt_percent'}, axis=1)
data_grouped_by_family_status['perсent_of_total'] = data_grouped_by_family_status['count'] / data['family_status'].count()*100
data_grouped_by_family_status.round(1).sort_values('debt_percent').reset_index(drop=True) #сортировка таблицы по % должников 

,family_status,count,sum,debt_percent,perсent_of_total
0,вдовец / вдова,951,63,6.6,4.5
1,в разводе,1189,84,7.1,5.6
2,женат / замужем,12261,927,7.6,57.5
3,гражданский брак,4134,385,9.3,19.4
4,не женат / не замужем,2796,273,9.8,13.1


**Вывод:** более половины заёмщиков составляют люди в официальном браке - 57,5%. Среди них должники составляют 7,6%.<p>Остальные группы можно разделить на 2 части:</p><ol><li>потерявшие супруга: вдовец/вдова и разведённые;</li><li>не состоящие в официальном браке: гражданский брак и не женат/не замужем;</li></ol>Среди заёмщиков, потерявших супруга доля должников ниже (6,6-7,1%), чем среди не состоящих в браке (9,3-9,8%). 

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

<p>План:</p>
Используя ранее применёную категоризацию по уровню дохода:</p>
<ul><li>0–30000 — `'E'`;</li>
<li>30001–50000 — `'D'`;</li>
<li>50001–200000 — `'C'`;</li>
<li>200001–1000000 — `'B'`;</li>
<li>1000001 и выше — `'A'`.</li></ul></p><p>На основе данной категоризации рассчитать количество заёмщиков и % должников в каждой категории.</p>

**Рассчет количества заёмщиков и % должников в каждой категории** 

In [28]:
data_grouped_by_total_income_cat = pd.pivot_table(data, index=['total_income_category'], values=['debt'], aggfunc=['count', 'sum', 'mean'])
data_grouped_by_total_income_cat = data_grouped_by_total_income_cat.reset_index().droplevel(1, axis=1)
data_grouped_by_total_income_cat['mean'] = data_grouped_by_total_income_cat['mean']*100 #перевод в %
data_grouped_by_total_income_cat = data_grouped_by_total_income_cat.rename({'mean':'debt_percent'}, axis=1)
data_grouped_by_total_income_cat['perсent_of_total'] = data_grouped_by_total_income_cat['count'] / data['total_income'].count()*100
data_grouped_by_total_income_cat.round(1)

,total_income_category,count,sum,debt_percent,perсent_of_total
0,A,25,2,8.0,0.1
1,B,5014,354,7.1,23.5
2,C,15921,1353,8.5,74.6
3,D,349,21,6.0,1.6
4,E,22,2,9.1,0.1


**Вывод:** процент заёмщиков в группах B и C составляет 98,1% от общего количества. Группы A, D, E слишком малочисленны, значениями можно пренебречь. Таким образом: <br><ul><li>среди заёмщиков с доходом от 200 т.р. до 1 млн 7,1% должников</li><li>среди заёмщиков с доходом от 50 т.р. до 200 т.р. 8.5% должников - чуть больше.</li></ul>

#### Как разные цели кредита влияют на его возврат в срок?

<p>План:</p>
<p>Используя ранее предобработанные данные о цели кредитования, рассчитать % долга для каждой категории.<p>

**Расчет**

In [29]:
data_grouped_by_purpose_cat = data.groupby('purpose_category').agg({'debt': ['count', 'sum', 'mean']}).reset_index().droplevel(0, axis=1)
data_grouped_by_purpose_cat['mean'] = data_grouped_by_purpose_cat['mean']*100 #перевод в %
data_grouped_by_purpose_cat = data_grouped_by_purpose_cat.rename({'mean':'debt_percent'}, axis=1)
data_grouped_by_purpose_cat['perсent_of_total'] = data_grouped_by_purpose_cat['count'] / data['purpose_category'].count()*100
data_grouped_by_purpose_cat.round(1).sort_values('debt_percent')


,,count,sum,debt_percent,perсent_of_total
1,операции с недвижимостью,10751,780,7.3,50.4
3,проведение свадьбы,2313,183,7.9,10.8
0,операции с автомобилем,4279,400,9.3,20.1
2,получение образования,3988,369,9.3,18.7


**Вывод:** Количество заёмщиком по цели соизмеримо, выборка по разным целям кредита сбалансированная. Максимальное количество должников среди заёмщиков с целями образования и операций с автомобилем - 9,3%. Минимальное - среди заёмщиков с целью недвижимости (7,3%) и свадьбы(7,9%).<p>Причины более высокой надёности заёмщиков с целями кредита "жильё" и "свадьба", предположительно таковы: <ul><li>сумма кредита на свадьбу небольшая, это позволяет взять кредит на комфортных условиях. Дополнительно облегчить его возврат могут денежные суммы, полученные молодоженами в качестве подарков.</li><li>Ипотека, как правило, большой и обременительный кредит, однако, в его залог часто идёт приобретаемая недвижимость. Таким образом, невозврат по итотеке чреват потерей всех вложений на жильё и утратой последнего.</li></ul>  
    

#### Возможные причины появления пропусков в исходных данных.

Выявлены пропуски в информации о стаже и общем доходе. Вероятно графы не были заполнены для заёмщиков, у которых отсутствует подтверждённый стаж и доход - это безработные, либо работающие неофициально. 

Аномалии в общем стаже - отрицательные значения - вероятно вызваны ошибкой обрабоки вводимых данных. Предположение: если стаж указан в виде нескольких интервалов, происходит ошибка при переводе интервалов в сумму отработанных дней. 

Рекомендация: проверить формат ввода информации о стаже и алгоритм обработки.

## Общий вывод.

#### Зависимость между количеством детей и возвратом кредита в срок 
<p>Для проведения исследования заёмщики были сгруппированы по количеству детей от 0 до 5. Исследование данных показало:</p>
<ol><li>Основную массу заёмщиков составляют бездетные(66,1%) и люди с одним ребёнком(22,5%) &ndash; суммарно 88,6%. Выборка по заёмщикам с 3 и более детьми мала, рассчетные данные могут непоказательны.</li><li>Доля должников среди бездетных составляет 7,5% - это ниже, чем среди заёмщиков с 1 и 2 детьми (9,2% и 9,5% соответственно). </li></ol>

#### Зависимость между семейным положением и возвратом кредита в срок
<p>В данных выявлены 5 значений семейного положения:</p><ul><li>вдовец / вдова</li><li>в разводе</li><li>женат / замужем</li><li>гражданский брак</li><li>не женат / не замужем</li></ul><p>Более половины заёмщиков составляют люди в официальном браке - 57,5%. Среди них должники составляют 7,6%.</p><p>Остальные группы логически можно разделить на 2 части:</p><ol><li>потерявшие супруга: вдовец/вдова и разведённые &ndash; 10,1%;</li><li>не состоящие в официальном браке: гражданский брак и не женат/не замужем &ndash; 32,5%.</li></ol><p>Данное деление корелирует с долей должников: cреди заёмщиков, оставщихся без супруга доля должников ниже (6,6-7,1%), чем среди не состоящих в браке (9,3-9,8%).</p>

#### Зависимость между уровнем дохода и возвратом кредита в срок
<p>Для исследования заёмщики были разделены на следующие категории по уровню дохода:</p><ul><li>0–30000 — `'E'`;</li>
<li>30001–50000 — `'D'`;</li>
<li>50001–200000 — `'C'`;</li>
<li>200001–1000000 — `'B'`;</li>
<li>1000001 и выше — `'A'`.</li></ul><p>Процент заёмщиков в c доходом от 50 т.р. до 1 млн.р.(группы B и C) составляет 98,1% от общего количества. Внутри этой группы: <br><ul><li>среди заёмщиков с доходом от 200 т.р. до 1 млн 7,1% должников</li><li>среди заёмщиков с доходом от 50 т.р. до 200 т.р. доля должников выше - 8.5%.</li></ul><p>Произведённые расчеты по остальным группам не показали линейной зависимости между возвратом кредита в срок и уровнем дохода. Т.е. надёжность заёмщиков не растёт пропорционально увеличениею дохода и, наоборот, со снижением дохода аккуратность выплат не повышается.</p>

#### Влияние цели кредита на его возврат в срок
<p>Анализ целей получения кредита позволил разделить заёмщиков на основные группы:</p><ul><li>операции с недвижимостью &ndash; 50,4% от общего количества заёмщиков;</li>
<li>проведение свадьбы &ndash; 10,8% ;</li>
<li>операции с автомобилем &ndash; 21,1%;</li>
<li>получение образования &ndash; 18,7%.</li></ul><p>
    Максимальное количество должников среди заёмщиков с целями образования и операций с автомобилем - 9,3%. Минимальное - среди заёмщиков с целью недвижимости (7,3%) и свадьбы(7,9%).<p>Причины более высокой надёности заёмщиков с целями кредита "жильё" и "свадьба", предположительно таковы: <ul><li>сумма кредита на свадьбу небольшая, это позволяет взять кредит на комфортных условиях. Дополнительно облегчить его возврат могут денежные суммы, полученные молодоженами в качестве подарков.</li><li>Ипотека, как правило, большой и обременительный кредит, однако, в его залог часто идёт приобретаемая недвижимость. Таким образом, невозврат по итотеке чреват потерей всех вложений на жильё и утратой последнего.</li></ul>  

#### Рекомендации
<p>По каждому из признаков для расчета надёжности можно применять следующие коэффициенты:<ul><li>Повышающий:</li>
<ul><li>бездетные;</li><li>с распавшейся семьёй(вдовец/вдова, в разводе);<li>с доходом 30-50т.р.;</li><li>с целью жилищного кредования.</li></ul>
<li>Понижающий:</li>
<ul><li>с детьми;</li><li>не состоящие в официальном браке(одинокие, гражданский брак);<li>с доходом 50-200т.р.;</li><li>с целью кредитования: операции с автомобилем и образование.</li></ul></ul><p>Колебания % должников по каждому из исследуемых признаков выявило разницу всего в 2-3 п.п. Поэтому повышающий или понижающий коэффициент по этим признакам должен иметь соответствующие невысокое влияние.</p>

***Пропуски в данных***
<p>В данных были выявлены пропуски в данных о стаже и доходе: у людей с отсутствовавшим значением стажа не был указан и доход. Данная ситуация может быть связана с тем, что люди не указываю доход и стаж, которые не могут подтвердить доход документально. </p><p><i>Рекомендация:</i> уточнить, есть ли возможность у заёмщика подтвердить доход и занятость альтернативными документами: выпиской с банковских счетов, справкой в свободной форме работодателя или другими.</p>

***Аномальные значения***
<p>В данных о стаже были обнаружены отрицательные значения. Вероятно это вызвано ошибкой обрабоки вводимых данных.</p><p><i>Предположение:</i> если стаж указан в виде нескольких интервалов, происходит ошибка при переводе интервалов в сумму отработанных дней. </p><p>Среди типов занятости "пенсионер" и "безработный" выявлен вредний стаж, в 100 и более раз превышающий стаж по остальным типам. Это так же указывает на ошибку в обработке данных о стаже.</p><p><i>Рекомендация:</i> проверить формат ввода информации о стаже и алгоритм обработки.</p>